In [2]:
import pandas as pd
import numpy as np
import os
#import cv2
import timm
import time
import torch
import torchvision
import skimage
#import wandb
#import hiplot
#import neptune.new as neptune
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from tqdm import tqdm
#from PIL import Image
#from torchsummary import summary
#from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
#Define DataSet class for the use of DataLoader class

class MaskDataset(Dataset):
    def __init__(self,labeldir,imgdir,transform):
        self.labeldata=pd.read_csv(labeldir)
        self.rootdir= imgdir
        self.transform=transform
    def __getitem__(self,index):
        img_path=os.path.join(self.rootdir,self.labeldata.iloc[index,0])
        image=skimage.io.imread(img_path)
        image=self.transform(image)
        label=torch.tensor(self.labeldata.iloc[index,1])
        return image,label
    def __len__(self):
        return self.labeldata.shape[0]

In [4]:
#Loading Data

trainlabel="D:\\python\\CIS473\\CIS-473-Machine-Learning\\mask_data\\labels\\train_label.csv"
traindir="D:\\python\\CIS473\\CIS-473-Machine-Learning\\mask_data\\traindata\\"
testlabel="D:\\python\\CIS473\\CIS-473-Machine-Learning\\mask_data\\labels\\test_label.csv"
testdir="D:\\python\\CIS473\\CIS-473-Machine-Learning\\mask_data\\testdata\\"

#Data Augmentation - # Hyperparameter
transform=transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
])

batch_size=1
traindata=MaskDataset(trainlabel,traindir,transform)
train=DataLoader(dataset=traindata,batch_size=batch_size,shuffle=True)
testdata=MaskDataset(testlabel,testdir,transform)
test=DataLoader(dataset=testdata,batch_size=batch_size,shuffle=True)
iter(train).next()[0].shape

4999

In [13]:
# Define Model
device=torch.device("cuda")

res34 = timm.create_model('resnet34')
res34.fc=nn.Linear(512,2)
res34.add_module("Sig",nn.Softmax())
res34 = res34.to(device)

# Loss Function - Hyperparameter
LossFunction = nn.CrossEntropyLoss()


#print(res34)
#summary(res34,(3,256,256))

class ConvolutionNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(            
            nn.Conv2d(3,20,3), #254
            nn.BatchNorm2d(20),
            nn.ReLU(),
            nn.Conv2d(20,50,3,stride=2), #127
            nn.BatchNorm2d(50),
            nn.ReLU(),
            nn.Conv2d(50,150,4), #124
            nn.BatchNorm2d(150),
            nn.ReLU(),
            nn.Conv2d(150,250,3,stride=2),#62
            nn.BatchNorm2d(250),
            nn.ReLU(),
            nn.Conv2d(250,400,3), #60
            nn.BatchNorm2d(400),
            nn.ReLU(),
            nn.Conv2d(400,512,3,stride=2), #29
            nn.BatchNorm2d(512),
            nn.ReLU(),
        )
        self.feature_size=512*29*29
        self.linear = nn.Sequential(
            nn.Linear(self.feature_size,200),
            nn.ReLU(),
            nn.Linear(200,500),
            nn.ReLU(),
            nn.Linear(500,1000),
            nn.ReLU(),
            nn.Linear(1000,500),
            nn.ReLU(),
            nn.Linear(500,100),
            nn.ReLU(),
            nn.Linear(100,2),
        )
    def forward(self,x_train):
        out=self.conv(x_train)
        out=out.view(batch_size,self.feature_size)
        out=self.linear(out)
        return out
    
conv=ConvolutionNet().to(device)

In [17]:
#Training Model

nepochs=10
#Learning Rate - # Hyperparameter
learning_rate=0.01
loss_history = []

#Optimization Method # Hyperparameter
optimizer = optim.SGD(conv.parameters(), lr=learning_rate)

for epoch in tqdm(range(nepochs),desc="Epoch",colour="#26B69C"):
    for data,label in tqdm(train,colour="#26B69C",mininterval=30):
        
        optimizer.zero_grad()
        data = data.to(device)
        label=label.to(device)
        out = conv(data)
        #print(out,label)
        loss = LossFunction(out,label)
        loss.backward()
        optimizer.step()
        loss_history.append(loss.item())
    print(f"Epoch {epoch}: loss: {loss.item()}")

Epoch:  10%|███████▌                                                                   | 1/10 [02:05<18:45, 125.08s/it]

Epoch 0: loss: 1.1920928244535389e-07



Epoch:  20%|███████████████                                                            | 2/10 [04:09<16:35, 124.45s/it]

Epoch 1: loss: 9.500529267825186e-05



Epoch:  30%|██████████████████████▌                                                    | 3/10 [06:13<14:30, 124.41s/it]

...

Epoch 7: loss: 0.0



Epoch:  90%|███████████████████████████████████████████████████████████████████▌       | 9/10 [18:34<02:03, 123.38s/it]

Epoch 8: loss: 5.960462772236497e-07



Epoch: 100%|██████████████████████████████████████████████████████████████████████████| 10/10 [20:37<00:00, 123.71s/it]

Epoch 9: loss: 0.0


In [19]:
#Testing Model

correct=0
total=0
with torch.no_grad():
    for data,label in tqdm(test,colour="#26B69C",mininterval=20):
        data = data.to(device)
        label=label.to(device)
        if data.shape==torch.Size([1, 3, 256, 256]):
            out=conv(data)
            pred=torch.argmax(out)
            if pred==label:
                correct+=1
            total+=1
print("Accuracy:",round(correct/total,3))
print(correct,total)

100%|█████████████████████████████████████████████████████████████████████████████| 1999/1999 [00:18<00:00, 106.06it/s]

Accuracy: 0.857
1708 1992


In [20]:
#Save Model

FILE="D:\\python\\CIS473\\CIS-473-Machine-Learning\\mask_data\\models\\model.pth"
torch.save(conv,FILE)